In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import Dataset, DataLoader

### No extra weight on buy/sell

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# load data and preprocess
df = pd.read_csv("training_data.csv")
df = df.drop(columns=['date', 'title', 'label_7']) # no label_7 here
df['stock_id'] = LabelEncoder().fit_transform(df['stock'])
df = df.drop(columns=['stock'])
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

# features / target
X = df.drop(columns=['label_3']).values
y = LabelEncoder().fit_transform(df['label_3'])

# scale numerics
X = StandardScaler().fit_transform(X)

# split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

class StockDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 64
train_loader = DataLoader(StockDataset(X_train, y_train), batch_size, shuffle=True)
val_loader   = DataLoader(StockDataset(X_val,   y_val),   batch_size)

# the actual model
class MLP(nn.Module):
    def __init__(self, in_dim, hid_dim, num_classes=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hid_dim),
            nn.ReLU(),
            nn.Linear(hid_dim, hid_dim),
            nn.ReLU(),
            nn.Linear(hid_dim, num_classes)
        )
    def forward(self, x):
        return self.net(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(in_dim=X_train.shape[1], hid_dim=128, num_classes=3).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
epochs = 10

for ep in range(1, epochs+1):
    # training
    model.train()
    running_loss = 0.0
    for Xb, yb in train_loader:
        Xb, yb = Xb.to(device), yb.to(device)
        preds = model(Xb)
        loss  = criterion(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * Xb.size(0)
    train_loss = running_loss / len(train_loader.dataset)

    # validation
    model.eval()
    correct = 0
    with torch.no_grad():
        for Xb, yb in val_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            correct += (model(Xb).argmax(1) == yb).sum().item()
    val_acc = correct / len(val_loader.dataset)

    print(f"Epoch {ep}/{epochs} — Train Loss: {train_loss:.4f} — Val Acc: {val_acc:.4f}")

torch.save({
    'model_state_dict': model.state_dict(),
    'scaler':           StandardScaler().fit(df.drop(columns=['label_3'])),
    'label_encoder':    LabelEncoder().fit(df['label_3']),
    'stock_encoder':    LabelEncoder().fit(df['stock_id'])
}, "stock_label3_model.pth")


Epoch 1/10 — Train Loss: 0.7021 — Val Acc: 0.7541
Epoch 2/10 — Train Loss: 0.6880 — Val Acc: 0.7551
Epoch 3/10 — Train Loss: 0.6829 — Val Acc: 0.7567
Epoch 4/10 — Train Loss: 0.6772 — Val Acc: 0.7583
Epoch 5/10 — Train Loss: 0.6728 — Val Acc: 0.7589
Epoch 6/10 — Train Loss: 0.6669 — Val Acc: 0.7597
Epoch 7/10 — Train Loss: 0.6600 — Val Acc: 0.7608
Epoch 8/10 — Train Loss: 0.6529 — Val Acc: 0.7626
Epoch 9/10 — Train Loss: 0.6448 — Val Acc: 0.7651
Epoch 10/10 — Train Loss: 0.6372 — Val Acc: 0.7661


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

df = pd.read_csv("training_data.csv")
df = df.drop(columns=['date', 'title', 'label_3'])  # different label here
df['stock_id'] = LabelEncoder().fit_transform(df['stock'])
df = df.drop(columns=['stock'])
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

X = df.drop(columns=['label_7']).values
y = LabelEncoder().fit_transform(df['label_7'])

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)


class StockDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 64
train_loader = DataLoader(StockDataset(X_train, y_train), batch_size, shuffle=True)
val_loader   = DataLoader(StockDataset(X_val,   y_val),   batch_size)

# no changes to the model
class MLP(nn.Module):
    def __init__(self, in_dim, hid_dim, num_classes=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hid_dim),
            nn.ReLU(),
            nn.Linear(hid_dim, hid_dim),
            nn.ReLU(),
            nn.Linear(hid_dim, num_classes)
        )
    def forward(self, x):
        return self.net(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(in_dim=X_train.shape[1], hid_dim=128, num_classes=3).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
epochs = 10

for ep in range(1, epochs+1):
    model.train()
    running_loss = 0.0
    for Xb, yb in train_loader:
        Xb, yb = Xb.to(device), yb.to(device)
        preds = model(Xb)
        loss  = criterion(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * Xb.size(0)
    train_loss = running_loss / len(train_loader.dataset)

    model.eval()
    correct = 0
    with torch.no_grad():
        for Xb, yb in val_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            correct += (model(Xb).argmax(1) == yb).sum().item()
    val_acc = correct / len(val_loader.dataset)

    print(f"Epoch {ep}/{epochs} — Train Loss: {train_loss:.4f} — Val Acc: {val_acc:.4f}")

torch.save({
    'model_state_dict': model.state_dict(),
    'scaler':           scaler,
    'label_encoder_7':  LabelEncoder().fit(df['label_7']),
    'stock_encoder':    LabelEncoder().fit(df['stock_id'])
}, "stock_label7_model.pth")


Epoch 1/10 — Train Loss: 0.8398 — Val Acc: 0.6693
Epoch 2/10 — Train Loss: 0.8261 — Val Acc: 0.6709
Epoch 3/10 — Train Loss: 0.8203 — Val Acc: 0.6717
Epoch 4/10 — Train Loss: 0.8140 — Val Acc: 0.6739
Epoch 5/10 — Train Loss: 0.8063 — Val Acc: 0.6744
Epoch 6/10 — Train Loss: 0.7995 — Val Acc: 0.6779
Epoch 7/10 — Train Loss: 0.7923 — Val Acc: 0.6823
Epoch 8/10 — Train Loss: 0.7839 — Val Acc: 0.6814
Epoch 9/10 — Train Loss: 0.7756 — Val Acc: 0.6830
Epoch 10/10 — Train Loss: 0.7684 — Val Acc: 0.6856


As we can see hold takes a heavy preference.

In [10]:
df = pd.read_csv("training_data.csv")
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

In [15]:
df['label_3'].value_counts(), df['label_7'].value_counts()

(label_3
 hold    85037
 buy     14993
 sell    12821
 Name: count, dtype: int64,
 label_7
 hold    75290
 buy     20808
 sell    16753
 Name: count, dtype: int64)

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

df = pd.read_csv("training_data.csv")
df = df.drop(columns=['date', 'title', 'label_3'])
df['stock_id'] = LabelEncoder().fit_transform(df['stock'])
df = df.drop(columns=['stock'])
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

X = df.drop(columns=['label_7']).values
y = LabelEncoder().fit_transform(df['label_7'])

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# change weights
counts = Counter(y_train)
total  = len(y_train)
num_classes = len(counts)
weights = [ total/(num_classes * counts[i]) for i in range(num_classes) ]
class_weights = torch.tensor(weights, dtype=torch.float32)

class StockDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 64
train_loader = DataLoader(StockDataset(X_train, y_train), batch_size, shuffle=True)
val_loader   = DataLoader(StockDataset(X_val,   y_val),   batch_size)


class MLP(nn.Module):
    def __init__(self, in_dim, hid_dim, num_classes=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hid_dim),
            nn.ReLU(),
            nn.Linear(hid_dim, hid_dim),
            nn.ReLU(),
            nn.Linear(hid_dim, num_classes)
        )
    def forward(self, x):
        return self.net(x)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(in_dim=X_train.shape[1], hid_dim=128, num_classes=3).to(device)

# move weights onto the same device as model, if using GPU
class_weights = class_weights.to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
epochs = 10

for ep in range(1, epochs+1):
    model.train()
    running_loss = 0.0
    for Xb, yb in train_loader:
        Xb, yb = Xb.to(device), yb.to(device)
        preds = model(Xb)
        loss  = criterion(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * Xb.size(0)
    train_loss = running_loss / len(train_loader.dataset)

    model.eval()
    correct = 0
    with torch.no_grad():
        for Xb, yb in val_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            correct += (model(Xb).argmax(1) == yb).sum().item()
    val_acc = correct / len(val_loader.dataset)

    print(f"Epoch {ep}/{epochs} — Train Loss: {train_loss:.4f} — Val Acc: {val_acc:.4f}")

torch.save({
    'model_state_dict': model.state_dict(),
    'scaler':           scaler,
    'label_encoder_7':  LabelEncoder().fit(df['label_7']),
    'stock_encoder':    LabelEncoder().fit(df['stock_id'])
}, "stock_label7_model.pth")


Epoch 1/10 — Train Loss: 1.0679 — Val Acc: 0.4503
Epoch 2/10 — Train Loss: 1.0575 — Val Acc: 0.5133
Epoch 3/10 — Train Loss: 1.0494 — Val Acc: 0.5351
Epoch 4/10 — Train Loss: 1.0401 — Val Acc: 0.5559
Epoch 5/10 — Train Loss: 1.0297 — Val Acc: 0.4721
Epoch 6/10 — Train Loss: 1.0191 — Val Acc: 0.4916
Epoch 7/10 — Train Loss: 1.0082 — Val Acc: 0.5233
Epoch 8/10 — Train Loss: 0.9966 — Val Acc: 0.5201
Epoch 9/10 — Train Loss: 0.9857 — Val Acc: 0.5034
Epoch 10/10 — Train Loss: 0.9759 — Val Acc: 0.5058


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

df = pd.read_csv("training_data.csv")
df = df.drop(columns=['date', 'title', 'label_7'])
df['stock_id'] = LabelEncoder().fit_transform(df['stock'])
df = df.drop(columns=['stock'])
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

X = df.drop(columns=['label_3']).values
y = LabelEncoder().fit_transform(df['label_3'])

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# same addition as before
counts = Counter(y_train)
total  = len(y_train)
num_classes = len(counts)
weights = [ total/(num_classes * counts[i]) for i in range(num_classes) ]
class_weights = torch.tensor(weights, dtype=torch.float32)
# move to GPU if available later


class StockDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 64
train_loader = DataLoader(StockDataset(X_train, y_train), batch_size, shuffle=True)
val_loader   = DataLoader(StockDataset(X_val,   y_val),   batch_size)

class MLP(nn.Module):
    def __init__(self, in_dim, hid_dim, num_classes=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hid_dim),
            nn.ReLU(),
            nn.Linear(hid_dim, hid_dim),
            nn.ReLU(),
            nn.Linear(hid_dim, num_classes)
        )
    def forward(self, x):
        return self.net(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(in_dim=X_train.shape[1], hid_dim=128, num_classes=3).to(device)

class_weights = class_weights.to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
epochs = 10

for ep in range(1, epochs+1):
    model.train()
    running_loss = 0.0
    for Xb, yb in train_loader:
        Xb, yb = Xb.to(device), yb.to(device)
        preds = model(Xb)
        loss  = criterion(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * Xb.size(0)
    train_loss = running_loss / len(train_loader.dataset)

    model.eval()
    correct = 0
    with torch.no_grad():
        for Xb, yb in val_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            correct += (model(Xb).argmax(1) == yb).sum().item()
    val_acc = correct / len(val_loader.dataset)

    print(f"Epoch {ep}/{epochs} — Train Loss: {train_loss:.4f} — Val Acc: {val_acc:.4f}")

torch.save({
    'model_state_dict': model.state_dict(),
    'scaler':           scaler,
    'label_encoder_3':  LabelEncoder().fit(df['label_3']),
    'stock_encoder':    LabelEncoder().fit(df['stock_id'])
}, "stock_label3_model.pth")


Epoch 1/10 — Train Loss: 1.0607 — Val Acc: 0.6052
Epoch 2/10 — Train Loss: 1.0457 — Val Acc: 0.4986
Epoch 3/10 — Train Loss: 1.0357 — Val Acc: 0.5421
Epoch 4/10 — Train Loss: 1.0257 — Val Acc: 0.5551
Epoch 5/10 — Train Loss: 1.0134 — Val Acc: 0.5444
Epoch 6/10 — Train Loss: 0.9999 — Val Acc: 0.4964
Epoch 7/10 — Train Loss: 0.9865 — Val Acc: 0.5797
Epoch 8/10 — Train Loss: 0.9729 — Val Acc: 0.5645
Epoch 9/10 — Train Loss: 0.9582 — Val Acc: 0.5445
Epoch 10/10 — Train Loss: 0.9430 — Val Acc: 0.5782
